### CALFEWS Annual Ensemble Runs 

This script includes code to run CALFEWS in a simulation mode annual.

Each run starts on Oct-1st using historical stroage levels from Oct-1st 1995 to Oct-1st 2022. 

Historical Hydrology flows are passed through all the model runs. 

The results are stored in `Annual_Ensembles`

In [1]:
%load_ext cython

In [2]:
%%cython
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import sys
from configobj import ConfigObj
import json
from distutils.util import strtobool
import h5py
from calfews_src.model_cy cimport Model
from calfews_src.inputter_cy import Inputter
from calfews_src.scenario import Scenario
from calfews_src.util import *
from calfews_src.plotter import *
from calfews_src.visualizer import Visualizer
from datetime import datetime
from subprocess import run
import glob


In [3]:
#Extract custom functions
from Annual_Runs.post_processing.functions.get_CVP_hydro_gen import get_CVP_hydro_gen

In [4]:
# %% Read the input data files
input_data = pd.read_csv("Annual_Ensembles/input_flows/cord-sim_realtime.csv", index_col=0)
input_data.index = pd.to_datetime(input_data.index)

In [5]:
#%% Misc functions needed

def get_water_year(date):
    """
    Convert calendar date to water year.
    Water year runs from Oct 1 to Sep 30.
    """
    return date.to_series().apply(lambda x: x.year + 1 if x.month >= 10 else x.year)

In [7]:
for year in range(2001, 2006):
    print(year)
    
    #Create a list of water years
    input_data['WY'] = get_water_year(input_data.index)
    
    #Find the current water year subset 
    cur_year_data = input_data[input_data['WY'] == year]
    
    #Save the current file 
    cur_year_data.to_csv('Annual_Ensembles/input_flows/annual_sim_inflow.csv', index=True)
    
    #Create all the input files after adjusting the indices and lengths.  
    for all_years in range(1996,2024):
        cur_yr = input_data[input_data['WY'] == all_years]
        
        if len(cur_year_data.index) == 365:
            cur_yr = cur_yr.iloc[:365]
            cur_yr.index = cur_year_data.index #Adjust the index/dates
        else:
            if len(cur_yr.index) == 365:
                cur_yr = pd.concat([cur_yr, cur_yr.iloc[[-1]]]) #Duplicate the last row
                cur_yr.index = cur_year_data.index
            else: 
                cur_yr.index = cur_year_data.index
        
        #Print the shape of the dataframe
        src_file = f"Annual_Ensembles/input_flows/WY_{all_years}.csv"
        cur_yr.to_csv(src_file, index=True)
    
    
    #-------------------------------------------------------------------------#
    #Make changes to the base inflow file (Done once per year)
    json_file_path = 'calfews_src/data/input/base_inflows.json'
    
    # Read all lines
    with open(json_file_path, 'r') as file:
        lines = file.readlines()
        
    #Change 1 -- Year start
    lines[241] = f'\t  "CDEC_annual_ensembles": {cur_year_data.index[0].year}\n'
    
    #Change 2 -- Year end
    lines[342] = f'\t  "CDEC_annual_ensembles": {cur_year_data.index[-1].year}\n'
    
    #Change 3 -- Date State
    lines[443] = f'\t  "CDEC_annual_ensembles": "{cur_year_data.index[0].strftime("%#m/%#d/%Y")}"\n'
    
    #Change 4 -- Date End 
    lines[544] = f'\t  "CDEC_annual_ensembles": "{cur_year_data.index[-1].strftime("%#m/%#d/%Y")}"\n'
    
    # Write all lines back
    with open(json_file_path, 'w') as file:
        file.writelines(lines)
        
    #-------------------------------------------------------------------------#
    #Start running CALFEWS
    for all_years in range(1996, 2024):
        
        print(f"CALFEWS - {all_years}")
        #Copy the input source file
        source_file = f"Annual_Ensembles/input_flows/WY_{all_years}.csv"
        dest_file = r"Annual_Ensembles/input_flows/annual_sim_inflow.csv"
        shutil.copy(source_file, dest_file)
    
        # Copy the runtime parameters file
        source_file = r"Annual_Ensembles/runtime_params_annual_ensembles.ini"
        dest_file = r"runtime_params.ini"
        shutil.copy(source_file, dest_file)
    
        # Set up the results validation path
        results_folder = f"Annual_Ensembles/results/{year}/{all_years}"
        
        if os.path.exists(results_folder):
            # Empty the folder if it exists
            shutil.rmtree(results_folder)
            os.makedirs(results_folder)
        else:
            # Create the folder if it doesn't exist
            os.makedirs(results_folder)
    
        # Run the Python script using subprocess
        run(["python", "-W", "ignore", "run_main_cy.py", results_folder, "1", "1"])
        
        #-------------------------------------------------------------------------#
        #Compute the hydropower generation
        results_file = f"{results_folder}/results.hdf5"
        results_data = get_results_sensitivity_number_outside_model(results_file, '')

        #Subset to needed columns 
        results_data = results_data[['shasta_S', 'shasta_R', 'oroville_S', 'oroville_R', 'trinity_S', \
                                     'trinity_R', 'folsom_S', 'folsom_R', 'newmelones_S', 'newmelones_R', \
                                     'sanluisstate_S', 'sanluisfederal_S', 'trinity_diversions', 'delta_TRP_pump', 'caa_SNL_flow']]

        #Get the storage data set
        storage_input = results_data[['shasta_S', 'trinity_S', 'folsom_S', 'newmelones_S']].copy()
        storage_input.columns = ['Shasta', 'Trinity', 'Folsom', 'New Melones']
        storage_input['San Luis'] = results_data['sanluisstate_S'] + results_data['sanluisfederal_S']

        #Get the releases dataset
        release_input = results_data[['shasta_R', 'trinity_R', 'folsom_R', 'newmelones_R']].copy()
        release_input.columns = ['Shasta', 'Trinity', 'Folsom', 'New Melones']
        release_input.loc[:,'Diversions'] = results_data['trinity_diversions']
        release_input['San Luis'] =  storage_input['San Luis'].diff().mul(-1).fillna(0)
        release_input.loc[(release_input.index.month == 10) & (release_input.index.day == 1), 'San Luis'] = 0

        #Compute the hydropower generation 
        cvp_gen = get_CVP_hydro_gen(storage_input, release_input)


        #Compute San Luis Pumping
        san_luis_pump = cvp_gen[["San_Luis", "Oneill"]]
        san_luis_pump = -1*san_luis_pump.clip(upper=0) #Only save pumping values
        san_luis_pump = san_luis_pump.resample('MS').sum()

        #Compute the pumping at Tracy
        tracy_efficiency = 232.7 #kWh/AF
        tracy_pump = results_data['delta_TRP_pump']*tracy_efficiency/1000 #GWh
        tracy_pump = tracy_pump.resample('MS').sum()

        #Compute pumping at Dos Amigoes
        dos_amigos_ei = 135.6 #Kwh/AF
        dos_amigos_pump = results_data['caa_SNL_flow']*dos_amigos_ei/1000 #Gwh
        dos_amigos_pump =  dos_amigos_pump.resample('MS').sum()

        #Remove San Luis and Oneill and aggregate to monthly
        cvp_gen = cvp_gen.drop(["San_Luis", "Oneill", "CVP_Gen"], axis=1)  
        cvp_gen['CVP_Gen'] = cvp_gen.sum(axis=1)
        
        #Save the daily data
        daily_gen = cvp_gen
        
        cvp_gen = cvp_gen.resample('MS').sum()

        #Classify the values
        cvp_gen['San_Luis_Pump'] = san_luis_pump['San_Luis'] + san_luis_pump['Oneill']
        cvp_gen['Tracy_Pump'] = tracy_pump
        cvp_gen['Dos_Amigos'] = dos_amigos_pump
        cvp_gen['Project_Use'] = (cvp_gen['San_Luis_Pump'] + cvp_gen['Tracy_Pump'] + cvp_gen['Dos_Amigos'])
        cvp_gen['BR'] = cvp_gen['CVP_Gen'] - cvp_gen['Project_Use']
        cvp_gen['BR'] = cvp_gen['BR'].clip(lower=0)

        #Clear the repository
        shutil.rmtree(results_folder)
        os.makedirs(results_folder)

        #Save the value
        cvp_gen.to_csv(f"{results_folder}/CVP_Generation.csv", index=True)
        daily_gen.to_csv(f"{results_folder}/Daily_CVP_Generation.csv", index=True)

2001
CALFEWS - 1996
CALFEWS - 1997
CALFEWS - 1998
CALFEWS - 1999
CALFEWS - 2000
CALFEWS - 2001
CALFEWS - 2002
CALFEWS - 2003
CALFEWS - 2004
CALFEWS - 2005
CALFEWS - 2006
CALFEWS - 2007
CALFEWS - 2008
CALFEWS - 2009
CALFEWS - 2010
CALFEWS - 2011
CALFEWS - 2012
CALFEWS - 2013
CALFEWS - 2014
CALFEWS - 2015
CALFEWS - 2016
CALFEWS - 2017
CALFEWS - 2018
CALFEWS - 2019
CALFEWS - 2020
CALFEWS - 2021
CALFEWS - 2022
CALFEWS - 2023
2002
CALFEWS - 1996
CALFEWS - 1997
CALFEWS - 1998
CALFEWS - 1999
CALFEWS - 2000
CALFEWS - 2001
CALFEWS - 2002
CALFEWS - 2003
CALFEWS - 2004
CALFEWS - 2005
CALFEWS - 2006
CALFEWS - 2007
CALFEWS - 2008
CALFEWS - 2009
CALFEWS - 2010
CALFEWS - 2011
CALFEWS - 2012
CALFEWS - 2013
CALFEWS - 2014
CALFEWS - 2015
CALFEWS - 2016
CALFEWS - 2017
CALFEWS - 2018
CALFEWS - 2019
CALFEWS - 2020
CALFEWS - 2021
CALFEWS - 2022
CALFEWS - 2023
2003
CALFEWS - 1996
CALFEWS - 1997
CALFEWS - 1998
CALFEWS - 1999
CALFEWS - 2000
CALFEWS - 2001
CALFEWS - 2002
CALFEWS - 2003
CALFEWS - 2004
CALFEWS - 